## Intro

This notebook generates an xlsx report providing insights into your Twingate environment from Network Events across all Remote Networks.

## Required Python Dependencies

This notebook uses dataframe and polars in order to process large network event datasets and outputs a xlsx spreadsheet, it therefore requires some very specific python libraries to work.

In [1]:
# Install python dependencies required
%pip install xlsxwriter
%pip install pandas
%pip install polars

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/H

# Specify the Network Event Report to process

Change the cell below and specify the full path to the CSV extracted from your Admin Console along with the output xlsx file.

In [2]:
# specify the timezone you are in, it will become visible in the dataframe and can be used to filter activities based on localtime
local_tz = 'America/Los_Angeles'
full_console_report = '/Users/brendansapience/Downloads/input_report_from_admin_console.csv'
workbookName = '/Users/brendansapience/Downloads/output_insights_report.xlsx'


# Load up the parsing functions below

The functions in the following cell are used to generate the final report, they can be combined and used together as well to produce more insights as needed by Admins.

The cell below only loads functions in memory.

In [7]:
import json,requests,re
import pandas as pd
from types import SimpleNamespace
import datetime, time, logging, sys, base64, math
from urllib.request import urlopen
from json import load

################################################################

# Use the following functions to extract what you need

################################################################

# converts an admin console connector reports to a flattened normalized DF
def convert_admin_console_report_to_df(conn_output,tz=None):
    df = pl.read_csv(conn_output)
    
    df = df.rename({"device_id":"device.id","start_time" : "timestamp.readable","status" : "connection.error_message","resource_domain" : "resource.address","applied_rule" : "resource.applied_rule","bytes_received" : "connection.rx","bytes_transferred" : "connection.tx","resource_id" : "resource.id","remote_network" : "remote_network.name","remote_network_id" : "remote_network.id","protocol" : "connection.protocol","resource_port" : "connection.resource_port","resource_ip" : "connection.resource_ip","connector_id" : "connector.id","user": "user.email", "user_id": "user.id", "client_ip" : "connection.client_ip","connector" : "connector.name"})
    df = df.drop(['relays', 'relay_ips','relay_ports','end_time'])
    df = df.with_columns(pl.col("timestamp.readable").str.replace(r" UTC", ""))
    df = df.with_columns(pl.col('timestamp.readable').str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f").dt.convert_time_zone(tz))
    df = df.with_columns(pl.format("ConnectorId:{}", "connector.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("RemoteNetworkId:{}", "remote_network.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("ResourceId:{}", "resource.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("UserId:{}", "user.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("DeviceId:{}", "device.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.col("connection.error_message")).fill_nan('NORMAL')
    df = df.with_columns((pl.col("timestamp.readable").dt.strftime('%Y-%m-%d')).alias("timestamp.yymmdd"))
    # Jan 31 2024 - resource.address is empty when the resource definition is a cidr block and instead resource_ip os used
    # we replace empty values in resource.address with the ip instead
    df = df.with_columns(
        pl.col("resource.address").fill_null(pl.col("connection.resource_ip"))
    )

    return df

# converts ANALYTICS output into a flattened normalized DF (except for Relay info)
# it is obtained by running the following command:
# journalctl -u twingate-connector --since "X min ago" | grep "ANALYTICS" | sed 's/.* ANALYTICS//' | sed 'r/ /\ /g' > somefile
def convert_connector_output_to_df(conn_output,tz=None):
    df = pd.DataFrame()
    f = open(connector_log, "r")
    for line in f.readlines():
        data = json.loads(line)
        df1 = pd.json_normalize(data)
        df = pd.concat([df, df1])

    # adding a human readable timestamp to each line
    df['timestamp.readable'] = df['timestamp'].apply(epoch_to_date,args=(tz,))

    # object ids in Connector logs are the internal DB ids which are base64 decoded versions of API Ids.
    # converting internal Ids to API Ids in DF
    df['connector.id'] = df['connector.id'].apply(convert_id_to_api_id,args=("connector",))
    df['remote_network.id'] = df['remote_network.id'].apply(convert_id_to_api_id,args=("remotenetwork",))
    df['resource.id'] = df['resource.id'].apply(convert_id_to_api_id,args=("resource",))
    df['user.id'] = df['user.id'].apply(convert_id_to_api_id,args=("user",))
    df['device.id'] = df['device.id'].apply(convert_id_to_api_id,args=("device",))
    df[['connection.error_message']] = df[['connection.error_message']].fillna('NORMAL')
    # dropping relay info
    df = df.drop(columns=['relays','connection.cbct_freshness'])

    return df

def get_all_timezones():
    zoneinfo.available_timezones()

# returns the unique list of addresses connected to by users.
def get_address_activity(df,addr):
    logging.debug("getting list of activities for a given address.")
    return df.loc[df["resource.address"] == addr]

def get_resource_activity(df,resource):
    logging.debug("getting list of activities for a given resource.")
    return df.loc[df["resource.applied_rule"] == resource]

def get_endpoints_from_resource_activity(df,resource):
    logging.debug("getting list of activities for a given resource.")
    adf = df.loc[df["resource.applied_rule"] == resource]
    return adf["resource.address"].unique()

# Polars alternative function
def get_endpoints_from_resource_activity2(df,resource):
    logging.debug("getting list of activities for a given resource.")
    adf = df.filter(pl.col("resource.applied_rule") == resource)
    #connection.resource_ip
    ResAddresses = adf["resource.address"].unique()
    IpAddresses = adf["connection.resource_ip"].unique()
    return adf["resource.address"].unique(),adf["connection.resource_ip"].unique()

def get_unique_addresses(df):
    logging.debug("getting unique list of endpoints hit by clients.")
    return df["resource.address"].unique()

def get_unique_resources(df):
    logging.debug("getting unique list of resources hit by clients.")
    return df["resource.applied_rule"].unique()

def get_user_activity(df,user):
    logging.debug("getting all records for a given user.")
    return df.loc[df["user.email"] == user]

# Polars alternative function
def get_user_activity2(df,user):
    logging.debug("getting all records for a given user.")
    return df.filter(pl.col("user.email") == str(user))

def get_user_client_ips(df,user):
    logging.debug("getting all records for a given user.")
    df1 = df.loc[df["user.email"] == user]
    return df1["connection.client_ip"].unique()

# Polars alternative function
def get_user_client_ips2(df,user):
    logging.debug("getting all records for a given user.")
    df1 = df.filter(pl.col("user.email") == str(user))
    return df1["connection.client_ip"].unique()

def get_connector_ids(df):
    return df["connector.id"].unique()

def get_connector_names(df):
    return df["connector.name"].unique()

def get_rn_ids(df):
    return df["remote_network.id"].unique()

def get_rn_names(df):
    return df["remote_network.name"].unique()

def get_activities_for_remote_network(df,rnid):
    #s,dbid = convert_api_id_to_id(rnid)
    return df.loc[df["remote_network.id"] == rnid]

def get_activities_for_connector(df,connid):
    #s,dbid = convert_api_id_to_id(connid)
    return df.loc[df["connector.id"] == connid]

def get_activity_between_dates(df,before,after,localtz):
    # assuming before and after dates are in localtz, they need to be adjusted to UTC before comparison
    logging.debug("converting before date from string to datetime.")
    dt_before = datetime.datetime.strptime(before, "%Y-%m-%d %H:%M:%S.%f")

    logging.debug("converting after date from string to datetime.")
    dt_after = datetime.datetime.strptime(after, "%Y-%m-%d %H:%M:%S.%f")

    logging.debug("converting before date to UTC.")
    dt_before_utc = tz_to_utc(dt_before,localtz)

    logging.debug("converting after date to UTC.")
    dt_after_utc = tz_to_utc(dt_after,localtz)

    logging.debug("before date in UTC: "+str(dt_before_utc))
    logging.debug("after date in UTC: "+str(dt_after_utc))

    logging.debug("converting UTC dates to epoch.")
    before_epoch = date_to_epoch(str(dt_before_utc).replace("+00:00",""))
    logging.debug("before date in UTC to epoch: "+str(before_epoch))

    after_epoch = date_to_epoch(str(dt_after_utc).replace("+00:00",""))
    logging.debug("after date in UTC to epoch: "+str(after_epoch))

    #print("before:"+str(before_epoch)+" and after: "+str(after_epoch))
    return df[df['timestamp'].between(after_epoch, before_epoch)]

# return rows that have an actual error message
def get_errors(df):
    #return df[~df['connection.error_message'].isna()]
    return df[~df['connection.error_message'].str.startswith('NORMAL')]

def get_failures(df):
    if "event_type" in df:
        return df.loc[df["event_type"] == "failed_to_connect"]

# Polars alternative function
def get_errors2(df):
    return df.filter(pl.col("connection.error_message") != "NORMAL")

# Polars alternative function
def get_failures2(df):
    if "event_type" in df:
        return df.filter(pl.col("event_type") == "failed_to_connect")

def get_top_addresses(df):
    unique_addr_in_df = df["resource.address"].unique()
    print("address,count,rx,tx")
    for addr in unique_addr_in_df:

        total_for_addr = df.loc[df["resource.address"] == addr]
        Total_rx = total_for_addr['connection.rx'].sum()
        Total_tx = total_for_addr['connection.tx'].sum()

        print(str(addr)+","+str(len(total_for_addr))+","+str(Total_rx)+","+str(Total_tx))

def get_top_users(df):
    unique_users = df["user.email"].unique()

    print("user,count,rx,tx")
    for user in unique_users:

        total_for_user = df.loc[df["user.email"] == user]
        Total_rx = total_for_user['connection.rx'].sum()
        Total_tx = total_for_user['connection.tx'].sum()

        print(str(user)+","+str(len(total_for_user))+","+str(Total_rx)+","+str(Total_tx))


################################################################

# Ignore the following functions, they provide tooling for other useful functions

################################################################

# Check if a particular address matches a particular resource definition.
# returns True / False, matching string
def does_addr_match_res_definition(addr,resource_definition):
    regex_definition = resource_definition.replace(".","\.").replace("*",".*").replace("?",".?")
    x = re.search(regex_definition, addr)
    if x:
        return True,x[0]
    else:
        return False,""

# alternate method to extract IP Geo location info
def get_ip_info(addr=''):
    str_addr = str(addr)
    if str_addr == 'nan':
        return {}
    else:
        url = 'https://ipinfo.io/' + str_addr + '/json'
    res = urlopen(url)
    #response from url(if res==None then check connection)
    data = load(res)
    #will load the json response into data
    return data

# Events in the log show internal DB IDs as opposed to API IDs (visible in the Admin Console)
# the following function converts the DB ID to API ID
def convert_id_to_api_id(id,objecttype):
    if objecttype:
        encoded = base64.b64encode((objecttype.capitalize()+":"+str(id)).encode('ascii'))
        return encoded.decode("utf-8")
    else:
        return None

# Opposite of previous function, converts an API ID to DB ID
def convert_api_id_to_id(id):
    decoded = base64.b64decode(id)
    objname,dbid = decoded.decode("utf-8").split(":")
    return objname.lower(),dbid

# converts an epoch time to a human readable date
def epoch_to_date(timestamp,tz):
    s, ms = divmod(timestamp, 1000)  # (1236472051, 807)
    mydate = '%s.%03d' % (time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(s)), ms)
    utc_time = datetime.datetime.strptime(mydate, "%Y-%m-%d %H:%M:%S.%f")
    if tz != None:
        #logging.debug("tz passed as a parameter: "+tz)
        return str(utc_to_tz(utc_time,tz))
    else:
        #logging.debug("tz not passed as a parameter")
        return str(utc_time)

def utc_to_tz(utc_time,tz):
    utc = ZoneInfo('UTC')
    localtz = ZoneInfo(tz)
    utctime = utc_time.replace(tzinfo=utc)
    localtime = utctime.astimezone(localtz)
    return localtime

def tz_to_utc(a_time,tz):
    utc = ZoneInfo('UTC')
    localtz = ZoneInfo(tz)
    atime = a_time.replace(tzinfo=localtz)
    utc_time = atime.astimezone(utc)
    return utc_time

# converts a human readable date to epoch time
def date_to_epoch(adate):
    utc_time = datetime.datetime.strptime(adate, "%Y-%m-%d %H:%M:%S.%f")
    epoch_time = (utc_time - datetime.datetime(1970, 1, 1)).total_seconds()
    return epoch_time*1000

# returns a DF as CSV
def get_df_as_csv(df):
    return df.to_csv(index=True)

def initiate_logging():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    root.addHandler(handler)

def get_state(ip):
    if ip in ip_info_dict and str(ip) != "nan":
        data = ip_info_dict[ip]
        if 'message' in data:
            if 'bogon' in data['message']:
                return("private")
            #else:
            #    print(data['message'])
        else:
            return(data['state_code'])
    else:
        return("IP not in dict: "+str(ip))

def get_country(ip):
    if ip in ip_info_dict and str(ip) != "nan":
        data = ip_info_dict[ip]
        if 'message' in data:
            if 'bogon' in data['message']:
                return("private")
            #else:
            #    print(data['message'])
        else:
            return(data['country_code3'])
    else:
        return("IP not in dict: "+str(ip))

def get_city(ip):
    if ip in ip_info_dict and str(ip) != "nan":
        data = ip_info_dict[ip]
        if 'message' in data:
            if 'bogon' in data['message']:
                return("private")
            #else:
            #    print(data['message'])
        else:
            return(data['city'])
    else:
        return("IP not in dict: "+str(ip))

# initiate logging (no need to change this)
initiate_logging()




# Generate the final (XLSX) Report

You can customize the output report below in many ways:

* Change the title of each Tab
* Activate or deactivate sections of the report

In [8]:
import polars as pl
import time
import json,requests,re
from types import SimpleNamespace
import datetime, time, logging, sys, base64, math
from zoneinfo import ZoneInfo
from urllib.request import urlopen
from json import load
from xlsxwriter import Workbook

doUserIPInfo = True # Export User/IP Info
userIPSummaryTitle = "User IP Details" # Title for Workbook Tab

doUserActInfo = True # Export User Activity Details
userActTitle = "User Activity Details" # Title for Workbook Tab

doResInfo = True # Export Full Resource List
doResTitle = "Full Resource List" # Title for Workbook Tab

doMatchList = True # Export Match List for Wildcard Resources
doMatchTitle = "Resource Matching List" # Title for Workbook Tab

doConnectorsStats = True # Export Overall Connectors Stats
doConnectorsStatsTitle = "Connector Activities" # Title for Workbook Tab

doConnectorsStatsPerConnector = True # Export Per Connectors Stats

doConnErrorsStats = True # Export Connection Error Stats
doConnErrorsStatsTitle = "Connection Errors" # Title for Workbook Tab

doDNSErrorsStats = True # Export DNS Error Stats
doDNSErrorsStatsTitle = "DNS Errors" # Title for Workbook Tab

# initiate logging (no need to change this)
initiate_logging()
startTime = time.time()

# Convert entire report output to DataFrame for processing.
#local_tz = 'America/Los_Angeles'
print("Converting imported report to DataFrame")
df = convert_admin_console_report_to_df(full_console_report,local_tz)
print("Report has been successfully converted.")

if (doUserIPInfo or doUserActInfo):
    unique_users = df["user.email"].unique()

# End User Public IP tab
if (doUserIPInfo):
### Get User IP Info
    print("extracting User Public IPs")

    user_ip_info_data = {
        "user":[],
        "ipinfo":[],
    }

    user_ip_info_df = pl.DataFrame(user_ip_info_data, schema={"user":pl.String, "ipinfo":pl.String})

    for user in unique_users:
        ips = get_user_client_ips2(df,user)
        iplist_with_geo = []
        for ip in ips:
            iplist_with_geo.append(str(ip))
            ip_info_for_user = ",".join(iplist_with_geo)

        temp_ip_summary_data = { "user": user, "ipinfo":ip_info_for_user }
        temp_ip_summary_df = pl.DataFrame(temp_ip_summary_data, schema={"user":pl.String, "ipinfo":pl.String})
        user_ip_info_df = pl.concat([user_ip_info_df,temp_ip_summary_df])

    user_ip_info_df = user_ip_info_df.rename({"user":"User Email","ipinfo" : "Public IPs"})

# End User activity tab
if (doUserActInfo):
    user_activity_data = {
        "user":[],
        "numconn":[],
        "numerr":[],
        "txbytes":[],
        "rxbytes":[]
    }
    user_activity_df = pl.DataFrame(user_activity_data, schema={"user":pl.String, "numconn":pl.Int32, "numerr":pl.Int32, "txbytes":pl.Int64, "rxbytes": pl.Int64})
    for user in unique_users:
        user_act = get_user_activity2(df,user)
        user_err = get_errors2(user_act)
        total_tx = user_act['connection.tx'].sum()
        total_rx = user_act['connection.rx'].sum()
        temp_act_data = { "user": user, "numconn":len(user_act), "numerr": len(user_err), "txbytes":total_tx, "rxbytes":total_rx }
        temp_act_df = pl.DataFrame(temp_act_data, schema={"user":pl.String, "numconn":pl.Int32, "numerr":pl.Int32, "txbytes":pl.Int64, "rxbytes": pl.Int64})
        user_activity_df = pl.concat([user_activity_df, temp_act_df])

    user_activity_df = user_activity_df.rename({"user":"User Email","numconn" : "Nb of Connections","numerr" : "Nb of Errors","txbytes" : "TX (bytes)","rxbytes":"RX (bytes)"})

# Overall Resource performance tab
if (doResInfo):
    all_errors = get_errors2(df)
    unique_addr_in_df = df["resource.address"].unique()
    res_data = {
        "address":[],
        "resourcedef":[], #Feb 3 2025
        "resourceids":[], #Feb 3 2025
        "rnetworks":[], #Feb 3 2025
        "numconn":[],
        "numerrs":[],
        "failrate":[],
        "totaltx":[],
        "totalrx":[],
        "tcp":[],
        "udp":[],
        "icmp":[]
    }
    res_data_df = pl.DataFrame(res_data, schema={"address":pl.String, "resourcedef":pl.String, "resourceids":pl.String, "rnetworks":pl.String, "numconn":pl.Int32, "numerrs":pl.Int32, "failrate":pl.Float32, "totaltx":pl.Int64, "totalrx":pl.Int64, "tcp":pl.String, "udp":pl.String, "icmp":pl.String})
    
    print("extracting Error Report")
    # print("address,number of connections,number of errors,failure rate(%),total tx, total rx,TCP[Ports],UDP[Ports],ICMP[Ports]")
    # ## for each address in the error list, check how many records are in error and what the percentage of errors is
    for addr in unique_addr_in_df:
        if str(addr) != "nan":
            total_for_addr = df.filter(pl.col("resource.address") == addr)
            err_for_addr = all_errors.filter(pl.col("resource.address") == addr)

            if len(total_for_addr) > 0:
                percentage_failure = round(len(err_for_addr) / len(total_for_addr) * 100,2)
            else:
                percentage_failure = 100

            port_connected = []
            addr_activities = df.filter(pl.col("resource.address") == addr)
            
            # "resource_id" : "resource.id","remote_network" : "remote_network.name","remote_network_id" : "remote_network.id"
    
            # feb 3 2025 - resource definitions
            res_definitions = addr_activities["resource.applied_rule"].unique()
            res_ids = addr_activities["resource.id"].unique()
            rnetwork_names = addr_activities["remote_network.name"].unique()

            res_def_list=[]
            for definition in res_definitions:
                res_def_list.append(definition)
            res_def_list_s=";".join(str(x) for x in res_def_list)
            
            res_id_list=[]
            for definition in res_ids:
                res_id_list.append(definition)
            res_id_list_s=";".join(str(x) for x in res_id_list)
            
            res_rnetwork_list=[]
            for definition in rnetwork_names:
                res_rnetwork_list.append(definition)
            res_rnetwork_list_s=";".join(str(x) for x in res_rnetwork_list)
            # Feb 3 2025 end
            
            total_tx = addr_activities['connection.tx'].sum()
            total_rx = addr_activities['connection.rx'].sum()

            addr_activities_protocols = addr_activities["connection.protocol"].unique()
            addr_activities_icmp = addr_activities.filter(pl.col("connection.protocol") == "icmp")
            addr_activities_tcp = addr_activities.filter(pl.col("connection.protocol") == "tcp")
            addr_activities_udp = addr_activities.filter(pl.col("connection.protocol") == "udp")

            ports_tcp = addr_activities_tcp["connection.resource_port"].unique()
            ports_udp = addr_activities_udp["connection.resource_port"].unique()
            ports_icmp = addr_activities_icmp["connection.resource_port"].unique()

            tcp_port_list = []
            udp_port_list = []
            icmp_port_list = []
            for port in ports_tcp:
                tcp_port_list.append(port)
            for port in ports_udp:
                udp_port_list.append(port)
            for port in ports_icmp:
                icmp_port_list.append(port)
            tcp_port_list_s=";".join(str(x) for x in tcp_port_list)
            udp_port_list_s=";".join(str(x) for x in udp_port_list)
            icmp_port_list_s=";".join(str(x) for x in icmp_port_list)
            
            res_data_temp_df = pl.DataFrame(res_data, schema={"address":pl.String, "resourcedef":pl.String, "resourceids":pl.String, "rnetworks":pl.String, "numconn":pl.Int32, "numerrs":pl.Int32, "failrate":pl.Float32, "totaltx":pl.Int64, "totalrx":pl.Int64, "tcp":pl.String, "udp":pl.String, "icmp":pl.String})


            temp_res_data = {"address":str(addr),"resourcedef":str(res_def_list_s),"resourceids":str(res_id_list_s),"rnetworks":str(res_rnetwork_list_s), "numconn":len(total_for_addr), "numerrs":len(err_for_addr), "failrate":percentage_failure, "totaltx":total_tx, "totalrx":total_rx, "tcp":tcp_port_list_s, "udp":udp_port_list_s, "icmp":icmp_port_list_s}
            temp_res_df = pl.DataFrame(temp_res_data, schema={"address":pl.String, "resourcedef":pl.String, "resourceids":pl.String, "rnetworks":pl.String, "numconn":pl.Int32, "numerrs":pl.Int32, "failrate":pl.Float32, "totaltx":pl.Int64, "totalrx":pl.Int64, "tcp":pl.String, "udp":pl.String, "icmp":pl.String})
            res_data_df = pl.concat([res_data_df, temp_res_df])

    res_data_df = res_data_df.rename({"address":"Resource Address","resourcedef":"Resource Definitions","resourceids":"Resource IDs","rnetworks":"Remote Networks","numconn" : "Nb of Connections","numerrs" : "Nb of Errors","totaltx" : "TX (bytes)","totalrx":"RX (bytes)","failrate":"Failure Rate (%)"})

# Resource matching list
if (doMatchList):
    # get the list of unique resource definitions that have caught traffic (with the list of actual addresses hit for each resource)
    print("extracting Resource Matching Report")
    unique_resources = df["resource.applied_rule"].unique()
    match_data = {
        "resource":[],
        "matches":[]
    }
    match_data_df = pl.DataFrame(match_data, schema={"resource":pl.String, "matches":pl.String})
    for r in unique_resources:
        addresses_for_resource,ips_for_resource = get_endpoints_from_resource_activity2(df,r)
        
        # if the list of unique addresses returns only 1 address and said address is None, then the source of truth should be IPs and not FQDNs
        if len(addresses_for_resource) == 1:
            addr_joined="; ".join(str(x) for x in addresses_for_resource)
            if addr_joined == "None":
                addr_joined="; ".join(str(x) for x in ips_for_resource)
        else:
            addr_joined="; ".join(str(x) for x in addresses_for_resource) 
                
        
        #if(len(addresses_for_resource) != 0):
        #    addr_joined="; ".join(str(x) for x in addresses_for_resource)
        #else:
        #    addr_joined = "N/A"
        temp_match_data = {"resource":r, "matches":addr_joined}
        temp_match_data_df = pl.DataFrame(temp_match_data, schema={"resource":pl.String, "matches":pl.String})
        match_data_df = pl.concat([match_data_df, temp_match_data_df])

    match_data_df = match_data_df.rename({"resource":"Resource Definition","matches" : "Corresponding Addresses"})

# DNS Error tab
if(doDNSErrorsStats):
    print("extracting DNS Errors")

    connact_data = {
            "address":[],
            "nberrors":[]
      }

    dns_errors_df = pl.DataFrame(connact_data, schema={"address":pl.String, "nberrors":pl.Int64})
    activities_dnserror = df.filter(pl.col("connection.error_message") == "DNS_ERROR")
    unique_dns_failures = activities_dnserror['resource.address'].unique()

    for addr in unique_dns_failures:
        dns_ct = activities_dnserror.filter(pl.col("resource.address") == addr).select(pl.len()).item()

        temp_dns_errors = {"address":addr, "nberrors":dns_ct}
        temp_dns_errors_df = pl.DataFrame(temp_dns_errors, schema={"address":pl.String, "nberrors":pl.Int64})

        dns_errors_df = pl.concat([dns_errors_df, temp_dns_errors_df])

    dns_errors_df = dns_errors_df.rename({"address":"Resource Address","nberrors" : "Number of DNS Errors"})

# Connection Error tab
if(doConnErrorsStats):
    print("extracting Connection Errors")

    connact_data = {
        "address":[],
        "nberrors":[]
    }

    conn_errors_df = pl.DataFrame(connact_data, schema={"address":pl.String, "nberrors":pl.Int64})
    activities_connectionfailed = df.filter(pl.col("connection.error_message") == "CONNECTION_FAILED")
    unique_address_failures = activities_connectionfailed['resource.address'].unique()

    for addr in unique_address_failures:
        err_ct = activities_connectionfailed.filter(pl.col("resource.address") == addr).select(pl.len()).item()

        temp_conn_errors = {"address":addr, "nberrors":err_ct}
        temp_conn_errors_df = pl.DataFrame(temp_conn_errors, schema={"address":pl.String, "nberrors":pl.Int64})

        conn_errors_df = pl.concat([conn_errors_df, temp_conn_errors_df])

    conn_errors_df = conn_errors_df.rename({"address":"Resource Address","nberrors" : "Number of Connection Errors"})

# Overall Connector performance tab
if (doConnectorsStats):
    print("extracting Overall Connector Stats")
    unique_connectors = df["connector.name"].unique()
    connact_data = {
        "connectorname":[],
        "rnname":[],
        "totalactivities":[],
        "connsuccess":[],
        "connerror":[],
        "dnserror":[]
    }
    connector_data_df = pl.DataFrame(connact_data, schema={"connectorname":pl.String,"rnname":pl.String, "totalactivities":pl.Int64, "connsuccess":pl.Int64, "connerror":pl.Int64, "dnserror":pl.Int64})
    for connector in unique_connectors:

        conn_activities = df.filter(pl.col("connector.name") == connector)
        rn_name = conn_activities['remote_network.name'].unique()[0]
        conn_activities_normal = conn_activities.filter(pl.col("connection.error_message") == "NORMAL")
        conn_activities_connectionfailed = conn_activities.filter(pl.col("connection.error_message") == "CONNECTION_FAILED")
        conn_activities_dnserror = conn_activities.filter(pl.col("connection.error_message") == "DNS_ERROR")

        temp_res_data = {"connectorname":str(connector),"rnname":str(rn_name), "totalactivities":len(conn_activities),"connsuccess":len(conn_activities_normal),"connerror":len(conn_activities_connectionfailed),"dnserror":len(conn_activities_dnserror)}
        temp_res_df = pl.DataFrame(temp_res_data, schema={"connectorname":pl.String,"rnname":pl.String, "totalactivities":pl.Int64, "connsuccess":pl.Int64, "connerror":pl.Int64, "dnserror":pl.Int64})

        connector_data_df = pl.concat([connector_data_df, temp_res_df])

    connector_data_df = connector_data_df.rename({"connectorname":"Connector Name","totalactivities" : "Nb of Activities","rnname":"Remote Network Name","connsuccess":"Successful Connections","connerror":"Failed Connections","dnserror":"DNS Errors"})

# Per Connector performance tab
if (doConnectorsStatsPerConnector):
    print("extracting Per Connector Report")
    unique_connectors = df["connector.name"].unique()
    connact_data = {
        "date":[],
        "connectorname":[],
        "rnname":[],
        "totalactivities":[],
        "connsuccess":[],
        "connerror":[],
        "dnserror":[]
    }

    all_connector_data = {}
    for connector in unique_connectors:
        per_connector_data_df = pl.DataFrame(connact_data, schema={"date":pl.String,"connectorname":pl.String,"rnname":pl.String, "totalactivities":pl.Int64, "connsuccess":pl.Int64, "connerror":pl.Int64, "dnserror":pl.Int64})
        conn_activities = df.filter(pl.col("connector.name") == connector)
        rn_name = conn_activities['remote_network.name'].unique()[0]
        unique_days_of_Activities = conn_activities["timestamp.yymmdd"].unique()
        for day in unique_days_of_Activities:
            conn_day_activities = conn_activities.filter(pl.col("timestamp.yymmdd") == day)
            connection_normal = conn_day_activities.filter(pl.col("connection.error_message") == "NORMAL")
            connection_errors = conn_day_activities.filter(pl.col("connection.error_message") == "CONNECTION_FAILED")
            dns_errors = conn_day_activities.filter(pl.col("connection.error_message") == "DNS_ERROR")

            temp_res_data = {"date":str(day),"connectorname":str(connector),"rnname":str(rn_name), "totalactivities":len(conn_day_activities),"connsuccess":len(connection_normal),"connerror":len(connection_errors),"dnserror":len(dns_errors)}

            temp_res_df = pl.DataFrame(temp_res_data, schema={"date":pl.String,"connectorname":pl.String,"rnname":pl.String, "totalactivities":pl.Int64, "connsuccess":pl.Int64, "connerror":pl.Int64, "dnserror":pl.Int64})

            per_connector_data_df = pl.concat([per_connector_data_df, temp_res_df])

        per_connector_data_df = per_connector_data_df.rename({"connectorname":"Connector Name","totalactivities" : "Nb of Activities","rnname":"Remote Network Name","connsuccess":"Successful Connections","connerror":"Failed Connections","dnserror":"DNS Errors"})

        all_connector_data[connector] = per_connector_data_df

# Generate the XLSX Report with all tabs & data
with Workbook(workbookName) as wb:

    if(doUserIPInfo):
        print("writing IP information...")
        user_ip_info_df.write_excel(
            workbook=wb,
            worksheet=userIPSummaryTitle
        )
    if(doUserActInfo):
        print("writing activity summary...")
        user_activity_df.write_excel(
            workbook=wb,
            worksheet=userActTitle
        )

    if(doResInfo):
        print("writing resource summary...")
        res_data_df.write_excel(
            workbook=wb,
            worksheet=doResTitle
        )
    if(doMatchList):
        print("writing resource match summary...")
        match_data_df.write_excel(
            workbook=wb,
            worksheet=doMatchTitle
        )
    if(doDNSErrorsStats):
        print("wrinting DNS error summary...")
        dns_errors_df.write_excel(
            workbook=wb,
            worksheet=doDNSErrorsStatsTitle
      )

    if(doConnErrorsStats):
        print("wrinting Connection error summary...")
        conn_errors_df.write_excel(
            workbook=wb,
            worksheet=doConnErrorsStatsTitle
      )
    if(doConnectorsStats):
        print("writing connector summary...")
        connector_data_df.write_excel(
        workbook=wb,
        worksheet=doConnectorsStatsTitle
    )

    if(doConnectorsStatsPerConnector):
        print("writing per Connector stats summary...")
        #all_connector_data
        for key in all_connector_data:
            all_connector_data[key].write_excel(
                workbook=wb,
                worksheet=key
            )

    for WS in wb.worksheets():
        WS.autofit()
        WS.autofilter('A1:Z1000')

    endTime = time.time()
    timeDiff = endTime - startTime
    timeDiff = timeDiff * 10**3
    print("Report Generation done. Here is how long it took to generate: " + str(timeDiff) + "ms")

Converting imported report to DataFrame
Report has been successfully converted.
extracting User Public IPs
extracting Error Report
extracting Resource Matching Report
extracting DNS Errors
extracting Connection Errors
extracting Overall Connector Stats
extracting Per Connector Report
writing IP information...
writing activity summary...
writing resource summary...
writing resource match summary...
wrinting DNS error summary...
wrinting Connection error summary...
writing connector summary...
writing per Connector stats summary...
Report Generation done. Here is how long it took to generate: 98662.09888458252ms
